# 1) Importing Python stuff:

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import logging
logging.basicConfig(filename='GRESKE_spectra_fitting_BAYES.log')

In [3]:
# Scientific libraries
import numpy as np

# Import Pandas
import pandas as pd

# Astro
import astropy.io.fits as fits

# Graphic libraries

%matplotlib notebook
import matplotlib.pyplot as plt

#optional 3ML imports

from threeML import *
from threeML.io.file_utils import *

import os
import re
import sys
import traceback

from ipywidgets import *


WARNING RuntimeWarning: Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML


WARNING RuntimeWarning: Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML


WARNING RuntimeWarning: Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML



# 2) GRB list:

In [4]:
GRB_list = [('GRB 081118', 'GRB081118876', 2.58),
('GRB 081221', 'GRB081221681', 2.26),
('GRB 081222', 'GRB081222204', 2.77),
('GRB 090926B', 'GRB090926914', 1.24),
('GRB 100728B', 'GRB100728439', 2.106),
('GRB 100816A', 'GRB100816026', 0.8035),
('GRB 110213A', 'GRB110213220', 1.46),
('GRB 120326A', 'GRB120326056', 1.798),
('GRB 120729A', 'GRB120729456', 0.8),
('GRB 120811C', 'GRB120811649', 2.671),
('GRB 131011A', 'GRB131011741', 1.874),
('GRB 140606B', 'GRB140606133', 0.384),
('GRB 140808A', 'GRB140808038', 3.29),
('GRB 141028A', 'GRB141028455', 2.33),
('GRB 091020', 'GRB091020900', 1.71),
('GRB 100704A', 'GRB100704149', 3.6),
('GRB 130518A', 'GRB130518580', 2.488),
('GRB 120119A', 'GRB120119170', 1.728),
('GRB 081126', 'GRB081126899', 2.4),
('GRB 091208B', 'GRB091208410', 1.0633),
('GRB 100814A', 'GRB100814160', 1.44),
('GRB 100906A', 'GRB100906576', 1.727),
('GRB 110102A', 'GRB110102788', 2.5),
('GRB 111228A', 'GRB111228657', 0.7163),
('GRB 140508A', 'GRB140508128', 1.027),
('GRB 160509A', 'GRB160509374', 1.17),
('GRB 160625B', 'GRB160625945', 1.406)]

In [5]:
besties = [('GRB 081118', 'GRB081118876', 2.58),
('GRB 081221', 'GRB081221681', 2.26),
('GRB 081222', 'GRB081222204', 2.77),
('GRB 090926B', 'GRB090926914', 1.24),
('GRB 100728B', 'GRB100728439', 2.106),
('GRB 100816A', 'GRB100816026', 0.8035),
('GRB 110213A', 'GRB110213220', 1.46),
('GRB 120326A', 'GRB120326056', 1.798),
('GRB 120729A', 'GRB120729456', 0.8),
('GRB 120811C', 'GRB120811649', 2.671),
('GRB 131011A', 'GRB131011741', 1.874),
('GRB 140606B', 'GRB140606133', 0.384),
('GRB 140808A', 'GRB140808038', 3.29),
('GRB 141028A', 'GRB141028455', 2.33),
('GRB 091020', 'GRB091020900', 1.71),
('GRB 100704A', 'GRB100704149', 3.6),
('GRB 130518A', 'GRB130518580', 2.488),
('GRB 120119A', 'GRB120119170', 1.728)]

In [6]:
multies = [('GRB 081126', 'GRB081126899', 2.4),
('GRB 091208B', 'GRB091208410', 1.0633),
('GRB 100814A', 'GRB100814160', 1.44),
('GRB 100906A', 'GRB100906576', 1.727),
('GRB 110102A', 'GRB110102788', 2.5),
('GRB 111228A', 'GRB111228657', 0.7163),
('GRB 140508A', 'GRB140508128', 1.027),
('GRB 160509A', 'GRB160509374', 1.17),
('GRB 160625B', 'GRB160625945', 1.406)]

In [7]:
len(GRB_list), len(besties) + len(multies)

(27, 27)

# 3) Fit function:

In [ ]:
def GRB_fit(GRB_name):
    
    fit_results_dir = os.path.join(os.getcwd(),'fit_results_BAYESIAN')
    if(path_exists_and_is_directory(fit_results_dir)==False):
        if_directory_not_existing_then_make(fit_results_dir)
    
    with within_directory('prepared_pha_files'):
        det_b_erange = ('250-30000')
        det_n_erange = ('8.1-900')

        det = dets[0]
        with fits.open('%s_%s_time-resolved.pha'%(GRB_name,det)) as hdul:
            number_of_timebins = hdul[1].header['NAXIS2']

        def fdata(id):
            timebin_ogip = []
            for det in dets:
                ogip = OGIPLike(name=det, 
                                observation='%s_%s_time-resolved.pha'%(GRB_name,det),
                                background='%s_%s_time-resolved_bak.pha'%(GRB_name,det),
                                response='%s_%s_time-resolved.rsp'%(GRB_name,det),
                                spectrum_number=id)
                if (det == "b0" or det == "b1"):
                    ogip.set_active_measurements(det_b_erange)
                else:
                    ogip.set_active_measurements(det_n_erange,exclude=['30-35'])
                timebin_ogip.append(ogip)
            return(DataList(*timebin_ogip))


        def fmodel_band(id):
            ps = PointSource('source', spectral_shape=Band_grbm(), ra=0, dec=0)
            model = Model(ps)
            #priors
            model.source.spectrum.main.Band_grbm.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
            model.source.spectrum.main.Band_grbm.xc.prior = Log_uniform_prior(lower_bound=10, upper_bound=1e4)
            model.source.spectrum.main.Band_grbm.alpha.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
            model.source.spectrum.main.Band_grbm.beta.prior = Uniform_prior(lower_bound=-4.0, upper_bound=-1.6)
            return model
        
        
        def fmodel_cutoffpl(id):
            ps = PointSource('source', spectral_shape=Cutoff_powerlaw(), ra=0, dec=0)
            model = Model(ps)
            #priors
            model.source.spectrum.main.Cutoff_powerlaw.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
            model.source.spectrum.main.Cutoff_powerlaw.xc.prior = Log_uniform_prior(lower_bound=10, upper_bound=1e4)
            model.source.spectrum.main.Cutoff_powerlaw.index.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
            return model


        # fitting each timebin and writing final pha files out - for each timebin!

        bs_set = []
        bs_set_band = []
        bs_set_cutoffpl = []
        bs_set_results = []
        model_names = []
        
        dic_band = 1e10
        dic_cutoffpl = 1e10
        
        for id in range(number_of_timebins):
            
            try:
                bs_band = BayesianAnalysis(fmodel_band(id), fdata(id))
                multinest_samples_band = bs_band.sample_multinest(n_live_points=400, chain_name='chains_band_%d/fit-'%id)
                dic_band = bs_band.results.statistical_measures['DIC']
                bs_set_band.append(bs_band)
                
            except Exception as ex:
                    logging.exception('\n %s_%s'%(GRB_name,id))
                    pass
    
            try:
                bs_cutoffpl = BayesianAnalysis(fmodel_cutoffpl(id), fdata(id))
                multinest_samples_cutoffpl = bs_cutoffpl.sample_multinest(n_live_points=400, chain_name='chains_cutoffpl_%d/fit-'%id)
                dic_cutoffpl = bs_cutoffpl.results.statistical_measures['DIC']
                bs_set_cutoffpl.append(bs_cutoffpl)

            except Exception as ex:
                    logging.exception('\n %s_%s'%(GRB_name,id))
                    pass
            
            
            # writing DICs, just in case of emergency...
        
            dic_path = '/data29s/fermi/abacelj/GRB_correlations/GRB_builder/DIC_values.txt'
            if file_existing_and_readable(dic_path)==False:
                dic = open(dic_path,'w')
                dic.write('GRB_name\ttimebin\tDIC_band\tDIC_cutoffpl\n')
                
            else:
                dic = open(dic_path,'a')
            dic.write('%s\t%d\t%f\t%f\n'%(GRB_name,id,dic_band,dic_cutoffpl))
            dic.close()    

            
            # check which one has smaller DIC! and write that one to results!
            if dic_band >= dic_cutoffpl:
                
                name = 'cutoffpl'
                bs_set_results.append(bs_cutoffpl.results)
                bs_set.append(bs_cutoffpl)
                model_names.append(name)
                
            else:
                name = 'band'
                bs_set_results.append(bs_band.results)
                bs_set.append(bs_band)
                model_names.append(name)
                
            
    for id in range(number_of_timebins):
        bs_set_results[id].write_to(os.path.join(fit_results_dir,"%s_timebin%s_%s_fit_result.fits"%(GRB_name,id+1,name)),
                                    overwrite=True)
     
    results = (bs_set_band,bs_set_cutoffpl,bs_set,model_names)
    return results

# 3) Fit function 2:

In [ ]:
def GRB_fit_MULTIES(GRB_name,pulse):
        
    fit_results_dir = os.path.join(os.getcwd(),'fit_results_BAYESIAN_%d'%pulse)
    if(path_exists_and_is_directory(fit_results_dir)==False):
        if_directory_not_existing_then_make(fit_results_dir)
    
    with within_directory('prepared_pha_files_%d'%pulse):
        det_b_erange = ('250-30000')
        det_n_erange = ('8.1-900')

        det = dets[0]
        with fits.open('%s_%d_%s_time-resolved.pha'%(GRB_name,pulse,det)) as hdul:
            number_of_timebins = hdul[1].header['NAXIS2']

        def fdata(id):
            timebin_ogip = []
            for det in dets:
                ogip = OGIPLike(name=det, 
                                observation='%s_%d_%s_time-resolved.pha'%(GRB_name,pulse,det),
                                background='%s_%d_%s_time-resolved_bak.pha'%(GRB_name,pulse,det),
                                response='%s_%d_%s_time-resolved.rsp'%(GRB_name,pulse,det),
                                spectrum_number=id)
                if (det == "b0" or det == "b1"):
                    ogip.set_active_measurements(det_b_erange)
                else:
                    ogip.set_active_measurements(det_n_erange,exclude=['30-35'])
                timebin_ogip.append(ogip)
            return(DataList(*timebin_ogip))


        def fmodel_band(id):
            ps = PointSource('source', spectral_shape=Band_grbm(), ra=0, dec=0)
            model = Model(ps)
            #priors
            model.source.spectrum.main.Band_grbm.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
            model.source.spectrum.main.Band_grbm.xc.prior = Log_uniform_prior(lower_bound=10, upper_bound=1e4)
            model.source.spectrum.main.Band_grbm.alpha.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
            model.source.spectrum.main.Band_grbm.beta.prior = Uniform_prior(lower_bound=-4.0, upper_bound=-1.6)
            return model
        
        
        def fmodel_cutoffpl(id):
            ps = PointSource('source', spectral_shape=Cutoff_powerlaw(), ra=0, dec=0)
            model = Model(ps)
            #priors
            model.source.spectrum.main.Cutoff_powerlaw.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
            model.source.spectrum.main.Cutoff_powerlaw.xc.prior = Log_uniform_prior(lower_bound=10, upper_bound=1e4)
            model.source.spectrum.main.Cutoff_powerlaw.index.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
            return model


        # fitting each timebin and writing final pha files out - for each timebin!
       
        bs_set = []
        bs_set_band = []
        bs_set_cutoffpl = []
        bs_set_results = []
        model_names = []
        
        dic_band = 1e10
        dic_cutoffpl = 1e10

        
        
        for id in range(number_of_timebins):
            
            try:
                bs_band = BayesianAnalysis(fmodel_band(id), fdata(id))
                multinest_samples_band = bs_band.sample_multinest(n_live_points=400, chain_name='chains_band_%d/fit-'%id)
                dic_band = bs_band.results.statistical_measures['DIC']
                bs_set_band.append(bs_band)
                
            except Exception as ex:
                    logging.exception('\n %s_%d_%s'%(GRB_name,pulse,id))
                    pass
    
            try:
                bs_cutoffpl = BayesianAnalysis(fmodel_cutoffpl(id), fdata(id))
                multinest_samples_cutoffpl = bs_cutoffpl.sample_multinest(n_live_points=400, chain_name='chains_cutoffpl_%d/fit-'%id)
                dic_cutoffpl = bs_cutoffpl.results.statistical_measures['DIC']
                bs_set_cutoffpl.append(bs_cutoffpl)

            except Exception as ex:
                    logging.exception('\n %s_%d_%s'%(GRB_name,pulse,id))
                    pass
            
            
            # writing DICs, just in case of emergency...
        
            dic_path = '/data29s/fermi/abacelj/GRB_correlations/GRB_builder/DIC_values.txt'
            if file_existing_and_readable(dic_path)==False:
                dic = open(dic_path,'w')
                dic.write('GRB_name\ttimebin\tDIC_band\tDIC_cutoffpl\n')
                
            else:
                dic = open(dic_path,'a')
            dic.write('%s_%d\t%d\t%f\t%f\n'%(GRB_name,pulse,id,dic_band,dic_cutoffpl))
            dic.close()    

            
            # check which one has smaller DIC! and write that one to results!
            if dic_band >= dic_cutoffpl:
                
                name = 'cutoffpl'
                bs_set_results.append(bs_cutoffpl.results)
                bs_set.append(bs_cutoffpl)
                model_names.append(name)
                
            else:
                name = 'band'
                bs_set_results.append(bs_band.results)
                bs_set.append(bs_band)
                model_names.append(name)
                
            
    for id in range(number_of_timebins):
        bs_set_results[id].write_to(os.path.join(fit_results_dir,"%s_%d_timebin%s_%s_fit_result.fits"%(GRB_name,pulse,id+1,name)),
                                    overwrite=True)
     
    results = (bs_set_band,bs_set_cutoffpl,bs_set,model_names)
    return results

# 4) Plot data and fit_model function:

In [ ]:
def plot_data_and_model(bs_set,temp_dir,model_names):
    
    #temp_dir = 'spectra_data-model_plots_BAYESIAN'
    if(path_exists_and_is_directory(temp_dir)==False):
        if_directory_not_existing_then_make(temp_dir)

    with within_directory(temp_dir):
        bs_set.restore_median_fit()
        for index in range(len(bs_set)):
            try:
                try:
                    fig1 = display_spectrum_model_counts(bs_set[index], step=False, figsize = (12,12))
                except:
                    fig1 = display_spectrum_model_counts(bs_set[index], min_rate=-1000, step=False, figsize = (12,12))
                
                ymin, ymax = fig1.axes[0].set_ylim()
                if(ymin < 10e-20):
                    ymin = 10e-10
                    fig1.axes[0].set_ylim(bottom=ymin)
                
                fig1.savefig('%s_timebin%s_data_and_%s_model_plot.png'%(GRB_name,index+1,model_names[index])
                        ,bbox_inches="tight", frameon=True, overwrite=True)
                
                #create corner plot
                fig2 = bs_set.results.corner_plot_cc()
                fig2.savefig('%s_timebin%s_data_and_%s_model_plot_cornerplot.png'%(GRB_name,index+1,model_names[index])
                        ,bbox_inches="tight", frameon=True, overwrite=True)
                
                plt.close()
                plt.ioff()
                         
            except Exception as ex:
                logging.exception('\n %s_%s'%(GRB_name,index))
                pass

# 4) Plot data and fit_model function 2:

In [ ]:
def plot_data_and_model_MULTIES(bs_set,pulse,temp_dir,model_names):
    
    #temp_dir = 'spectra_data-model_plots_BAYESIAN_%d'%pulse
    if(path_exists_and_is_directory(temp_dir)==False):
        if_directory_not_existing_then_make(temp_dir)

    with within_directory(temp_dir):
        bs_set.restore_median_fit()
        for index in range(len(bs_set)):
            try:
                try:
                    fig1 = display_spectrum_model_counts(bs_set[index], step=False, figsize = (12,12))
                except:
                    fig1 = display_spectrum_model_counts(bs_set[index], min_rate=-1000, step=False, figsize = (12,12))
                
                ymin, ymax = fig1.axes[0].set_ylim()
                if(ymin < 10e-20):
                    ymin = 10e-10
                    fig1.axes[0].set_ylim(bottom=ymin)
                
                fig1.savefig('%s_%d_timebin%s_data_and_%s_model_plot.png'%(GRB_name,pulse,index+1,model_names[index])
                        ,bbox_inches="tight", frameon=True, overwrite=True)
                
                #create corner plot
                fig2 = bs_set.results.corner_plot_cc()
                fig2.savefig('%s_timebin%s_data_and_%s_model_plot_cornerplot.png'%(GRB_name,index+1,model_names[index])
                        ,bbox_inches="tight", frameon=True, overwrite=True)
                
                plt.close()
                plt.ioff()
                         
            except Exception as ex:
                logging.exception('\n %s_%d_%s'%(GRB_name,pulse,index))
                pass

# 5) MAIN - Collecting general GRB data and analysis:

In [ ]:
gbm_cat = FermiGBMBurstCatalog()

In [ ]:
os.getcwd()

In [ ]:
start_path = '/data29s/fermi/abacelj/GRB_correlations/GRB_builder'
os.chdir(start_path)

In [ ]:
for trig, GRB_name, z in besties:

    gbm_cat.query_sources(GRB_name)
    grb_info = gbm_cat.get_detector_information()[GRB_name]
    dets = grb_info['detectors']

    GRB_directory = "%s"%(GRB_name)     

    try:
        with within_directory(GRB_directory):
            bs_set_band, bs_set_cutoffpl, bs_set_fit, model_names_fit = GRB_fit(GRB_name)

    except Exception as ex:
        logging.exception('\n %s'%GRB_name)
        os.chdir(start_path)
        pass

    try:
        with within_directory(GRB_directory):
            plot_data_and_model(bs_set_fit, 'spectra_data_fitted-model_plots_BAYESIAN',model_names_fit)
    
    except Exception as ex:
        logging.exception('\n %s'%GRB_name)
        os.chdir(start_path)
        pass
    
    try:
        with within_directory(GRB_directory):
            model_names_band = []
            for i in range(len(model_names_fit)):
                model_names_band.append('band') 
            plot_data_and_model(bs_set_band, 'spectra_data-BAND_plots_BAYESIAN',model_names_band)
    
    except Exception as ex:
        logging.exception('\n %s'%GRB_name)
        os.chdir(start_path)
        pass
    
    try:
        with within_directory(GRB_directory):
            model_names_cutoffpl = []
            for i in range(len(model_names_fit)):
                model_names_cutoffpl.append('cutoffpl')
            plot_data_and_model(bs_set_cutoffpl, 'spectra_data-CUTOFFPL_plots_BAYESIAN',model_names_cutoffpl)
    
    except Exception as ex:
        logging.exception('\n %s'%GRB_name)
        os.chdir(start_path)
        pass

## next part is MULTIES...

In [ ]:
for trig, GRB_name, z in multies:
    
    gbm_cat.query_sources(GRB_name)
    grb_info = gbm_cat.get_detector_information()[GRB_name]
    dets = grb_info['detectors']
    
    GRB_directory = "%s"%(GRB_name)
    
    with within_directory(GRB_directory):
        number_of_pulses = 0
        for i in range(1,10):
            pulse_dir = 'prepared_pha_files_%d'%i
            if(path_exists_and_is_directory(pulse_dir)==True):
                number_of_pulses += 1

    for pulse in range(1,number_of_pulses+1):
    
        try:
            with within_directory(GRB_directory):
                bs_set_band, bs_set_cutoffpl, bs_set_fit, model_names_fit = GRB_fit_MULTIES(GRB_name)

        except Exception as ex:
            logging.exception('\n %s_%d'%GRB_name,pulse)
            os.chdir(start_path)
            pass
        
        try:
            with within_directory(GRB_directory):
                plot_data_and_model_MULTIES(bs_set_fit,pulse,'spectra_data_fitted-model_plots_BAYESIAN_%d'%pulse,model_names_fit)

        except Exception as ex:
            logging.exception('\n %s_%d'%GRB_name,pulse)
            os.chdir(start_path)
            pass
        
        try:
            with within_directory(GRB_directory):
                model_names_band = []
                for i in range(len(model_names_fit)):
                    model_names_band.append('band') 
                plot_data_and_model_MULTIES(bs_set_band,pulse,'spectra_data-BAND_plots_BAYESIAN_%d'%pulse,model_names_band)
                    
        except Exception as ex:
            logging.exception('\n %s_%d'%GRB_name,pulse)
            os.chdir(start_path)
            pass

        try:
            with within_directory(GRB_directory):
                model_names_cutoffpl = []
                for i in range(len(model_names_fit)):
                    model_names_cutoffpl.append('cutoffpl')
                plot_data_and_model_MULTIES(bs_set_cutoffpl,pulse,'spectra_data-CUTOFFPL_plots_BAYESIAN_%d'%pulse,model_names_cutoffpl)

        except Exception as ex:
            logging.exception('\n %s_%d'%GRB_name,pulse)
            os.chdir(start_path)
            pass